In [1]:
# Задание на повторение материала предыдущего семестра

In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [48]:
# Генерируем уникальный seed
my_code = "Лукьянчук"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [49]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [50]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

year              region  npg  birth_rate  death_rate    gdw  urbanization
0  1990  Republic of Adygea  1.9        14.2        12.3  84.66         52.42
1  1990          Altai Krai  1.8        12.9        11.1  80.24         58.07
2  1990         Amur Oblast  7.6        16.2         8.6  69.55         68.37
3  1990  Arkhangelsk Oblast  3.7        13.5         9.8  73.26         73.63
4  1990    Astrakhan Oblast  4.7        15.1        10.4  77.05         68.01

In [5]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [6]:
example_data.shape

(2315, 7)

In [7]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [8]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [9]:
train.head()

year                  region  npg  birth_rate  death_rate    gdw  \
2253  2016  Nizhny Novgorod Oblast -3.5        11.9        15.4  76.80   
1737  2010        Mari El Republic -2.5        12.7        15.2  57.62   
1603  2008         Udmurt Republic -0.7        13.3        14.0  56.33   
1324  2005            Penza Oblast -9.8         8.4        18.1  63.63   
1943  2012         Udmurt Republic  2.4        15.3        12.9  64.56   

      urbanization  
2253          79.5  
1737          63.0  
1603          70.4  
1324          65.6  
1943          68.9

In [10]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [11]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [12]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [13]:
sc_val

year     npg birth_rate death_rate       gdw urbanization  \
0    0.555556  0.1675   0.110638   0.669903  0.383145     0.740736   
1    0.888889  0.2575   0.225532   0.645631  0.638401     0.561346   
2    0.296296     0.2   0.059574   0.567961   0.71225     0.520754   
3     0.37037    0.12   0.059574   0.723301  0.716594     0.586094   
4    0.888889    0.61   0.485106   0.257282  0.439791     0.544324   
..        ...     ...        ...        ...       ...          ...   
458  0.407407    0.01   0.068085   0.946602  0.640834     0.550085   
459  0.074074  0.4275   0.255319   0.349515  0.782103     0.467199   
460       1.0   0.405   0.280851    0.42233   0.62172     0.761687   
461  0.148148    0.26    0.07234   0.466019  0.559861     0.536074   
462  0.888889   0.375   0.310638   0.514563  0.533623     0.569203   

                       region  
0               Moscow Oblast  
1                Kursk Oblast  
2              Lipetsk Oblast  
3              Bryansk Oblast  
4    Sakha (Yakutia) Republic  
..                        ...  
458              Pskov Oblast  
459          Chuvash Republic  
460           Sakhalin Oblast  
461              Tomsk Oblast  
462               Amur Oblast  

[463 rows x 7 columns]

In [14]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [15]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [16]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [17]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [18]:
y_labels

['urbanization']

In [19]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [20]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.018868623122076253
Lasso() 	 0.0270968859704158
Ridge() 	 0.018870202991669607
ElasticNet() 	 0.0270968859704158
KNeighborsRegressor() 	 0.010997108436099738
KNeighborsRegressor(n_neighbors=10) 	 0.011817969157962726
KNeighborsRegressor(n_neighbors=15) 	 0.012820538499059104
SVR(kernel='linear') 	 0.01903188400048227
SVR(kernel='poly') 	 0.011790660182904889
SVR() 	 0.011313453804013779
SVR(kernel='sigmoid') 	 36.94755088864023
DecisionTreeRegressor() 	 0.024123396851907165
DecisionTreeRegressor(criterion='friedman_mse') 	 0.024836820409221163
DecisionTreeRegressor(criterion='absolute_error') 	 0.02210668005094026


In [21]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [22]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.010664762152687031


In [23]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [24]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [25]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [26]:
x_train

year     npg birth_rate death_rate       gdw urbanization
0     0.962963  0.2925   0.234043   0.587379  0.682884      0.73157
1     0.740741  0.3175   0.268085    0.57767  0.349609     0.515517
2     0.666667  0.3625   0.293617   0.519417  0.327194     0.612413
3     0.555556   0.135   0.085106   0.718447   0.45404     0.549561
4     0.814815    0.44   0.378723   0.466019    0.4702     0.592772
...        ...     ...        ...        ...       ...          ...
1384  0.592593  0.3725   0.195745   0.412621  0.138662     0.723714
1385  0.740741  0.3325   0.306383   0.592233  0.384361     0.571821
1386  0.444444    0.17   0.106383   0.679612  0.377237     0.706953
1387  0.592593    0.17   0.119149    0.68932  0.421373     0.675265
1388  0.592593  0.3875   0.225532   0.359223  0.412511     0.271965

[1389 rows x 6 columns]

In [27]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')
LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)
MultinomialNB()


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)
KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')
SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [28]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.3235866146868561
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.11725306000252818
LogisticRegression(solver='saga') 	 0.03367618952299001
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.036865778380365574
LogisticRegression() 	 0.03367618952299001
MultinomialNB(alpha=0.0) 	 0.051232103312357946
MultinomialNB(alpha=0.5) 	 0.03857266180346608
MultinomialNB() 	 0.03089901318876623
KNeighborsClassifier() 	 0.3223973084834742
KNeighborsClassifier(n_neighbors=10) 	 0.26860343633885847
KNeighborsClassifier(n_neighbors=15) 	 0.2068968177581408
SVC(kernel='linear') 	 0.06256682901184414
SVC(kernel='poly') 	 0.6831480089103009
SVC() 	 0.17706218484357547
SVC(kernel='sigmoid') 	 8.342680225066974e-05
DecisionTreeClassifier() 	 0.6200772562078889
DecisionTreeClassifier(splitter='random') 	 0.5856983363278959
DecisionTreeClassifier(criterion='entropy') 	 0.6513302137580802
DecisionTreeCl

In [29]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

SVC(kernel='poly')

In [30]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.6823996927986256


In [31]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [32]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

year     npg birth_rate death_rate       gdw urbanization
0    0.962963  0.2925   0.234043   0.587379  0.682884      0.73157
1    0.740741  0.3175   0.268085    0.57767  0.349609     0.515517
2    0.666667  0.3625   0.293617   0.519417  0.327194     0.612413
3    0.555556   0.135   0.085106   0.718447   0.45404     0.549561
4    0.814815    0.44   0.378723   0.466019    0.4702     0.592772
..        ...     ...        ...        ...       ...          ...
458  0.592593    0.23   0.208511    0.68932  0.344396     0.804897
459  0.333333   0.275   0.093617   0.461165  0.296264     0.820086
460  0.777778   0.365   0.302128   0.524272  0.461859     0.789184
461  0.962963   0.705   0.421277        0.0  0.570287     0.231374
462  0.074074  0.4125   0.255319   0.378641   0.70252     0.609533

[2315 rows x 6 columns]

In [33]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [34]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.29501717139453104
0.24600626834472533
0.23892124763536562
0.23308709488173784


In [35]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.29501717139453104


In [36]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [37]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[ElasticNet(alpha=0.5, l1_ratio=0.75), KNeighborsRegressor(n_neighbors=15), SVR(kernel='linear'), SVR(kernel='sigmoid')]


In [38]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv")
data

water_level precipitation temperature humidity visibility wind weather  \
0          468,0           0,0        24,9     31,0      100,0  4,0     0,0   
1          140,0           0,0        27,9     22,0       20,0  1,0     0,0   
2          464,0           0,0        25,6     28,0       20,0  2,0     1,0   
3          467,0           0,0        26,7     26,0      100,0  3,0     0,0   
4          138,0           0,0        29,1     22,0       20,0  1,0     0,0   
...          ...           ...         ...      ...        ...  ...     ...   
1434       474,0           0,1         6,4     95,0        4,0  1,0     3,0   
1435        65,0           0,0         6,4     93,0       10,0  4,0     3,0   
1436       159,0           0,1         8,5     83,0       10,0  2,0     1,0   
1437        99,0           0,1         7,5     91,0       10,0  3,0     3,0   
1438       227,0           0,3         6,9     79,0       10,0  7,0     4,0   

     pressure fire wl_change temp_change pressure_change  
0       993,8  4,0      -3,0        -0,3            -0,4  
1      1004,1  4,0      -2,0         0,6            -1,0  
2      1001,2  4,0      -7,0        -1,8            -0,3  
3       992,3  4,0      -1,0         1,2            -1,3  
4      1001,3  4,0      -2,0         2,5            -1,3  
...       ...  ...       ...         ...             ...  
1434    999,9  4,0      -1,0         5,9             1,5  
1435    983,9  4,0      -1,0         5,2             3,1  
1436    969,7  4,0       1,0         1,4             1,9  
1437    995,4  4,0      -4,0         2,7             1,7  
1438    990,4  4,0      -1,0         1,3             1,7  

[1439 rows x 12 columns]

In [39]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['wl_change', 'wind', 'pressure_change', 'visibility', 'pressure']


In [40]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [41]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [42]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[LogisticRegression(penalty='l1', solver='saga'), LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'), KNeighborsClassifier(n_neighbors=25), LogisticRegression(l1_ratio=0.25, penalty='elasticnet', solver='saga'), KNeighborsClassifier()]


In [43]:
# Загрузим данные для задачи классификации
data = pd.read_csv("datasets/zoo2.csv")
data

animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0       turtle     0         0     1     0         0        1         0   
1    chameleon     0         0     1     0         0        0         0   
2       iguana     0         0     1     0         0        0         1   
3       lizard     0         0     1     0         0        0         1   
4        gecko     0         0     1     0         0        0         0   
5       python     0         0     1     0         0        0         1   
6          boa     0         0     1     0         0        0         1   
7        adder     0         0     1     0         0        0         1   
8    crocodile     0         0     1     0         0        1         1   
9    alligator     0         0     1     0         0        1         1   
10     gharial     0         0     1     0         0        1         1   
11       skink     0         0     1     0         0        0         0   
12     anchovy     0         0     1     0         0        1         0   
13    flounder     0         0     1     0         0        1         0   
14     halibut     0         0     1     0         0        1         0   
15    mackerel     0         0     1     0         0        1         0   
16   barracuda     0         0     1     0         0        1         1   
17      marlin     0         0     1     0         0        1         0   
18       trout     0         0     1     0         0        1         0   
19  salamander     0         0     1     0         0        1         0   
20       siren     0         0     1     0         0        1         0   
21   tree frog     0         0     1     0         0        1         0   
22   dart frog     0         0     1     0         0        1         0   
23   firebelly     0         0     1     0         0        1         0   
24   wart toad     0         0     1     0         0        1         0   
25    mosquito     0         0     1     0         1        0         0   
26      hornet     0         0     1     0         1        0         1   
27     cricket     0         0     1     0         0        0         0   
28      beetle     0         0     1     0         0        0         0   
29   butterfly     1         0     1     0         1        0         0   
30    palmetto     0         0     1     0         1        0         1   
31   cockroach     0         0     1     0         0        0         0   
32      mantis     0         0     1     0         0        0         1   
33   dragonfly     0         0     1     0         1        0         0   
34       aphid     0         0     1     0         0        0         0   
35      cicada     0         0     1     0         1        0         0   
36     antlion     0         0     1     0         1        0         0   
37     scallop     0         0     1     0         0        1         0   
38      spider     0         0     1     0         0        0         1   
39       snail     0         0     1     0         0        0         0   
40    silkworm     0         0     1     0         0        0         0   
41   jellyfish     0         0     1     0         0        1         0   
42       squid     0         0     1     0         0        1         0   

    toothed  backbone  breathes  venomous  fins  legs  tail  domestic  \
0         0         1         1         0     0     4     1         1   
1         1         1         1         0     0     4     1         1   
2         1         1         1         0     0     4     1         1   
3         1         1         1         0     0     4     1         0   
4         1         1         1         0     0     4     1         1   
5         1         1         1         1     0     0     1         0   
6         1         1         1         0     0     0     1         0   
7         1         1         1         1     0     0     1         0   
8         1         1         1         0    

In [44]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = 'class_type'
x_labels = random.sample(columns[1:-1], n_x)

print(x_labels)

['toothed', 'backbone', 'hair', 'legs', 'breathes', 'milk', 'predator', 'tail']


In [45]:
labels = x_labels.copy()
labels.append(y_label)
labels

['toothed',
 'backbone',
 'hair',
 'legs',
 'breathes',
 'milk',
 'predator',
 'tail',
 'class_type']

In [46]:
x_labels

['toothed', 'backbone', 'hair', 'legs', 'breathes', 'milk', 'predator', 'tail']

In [47]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [52]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [53]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[MultinomialNB(), SVC(kernel='poly'), KNeighborsClassifier(n_neighbors=20), SVC(kernel='linear'), LogisticRegression(penalty='none', solver='saga')]


In [54]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("datasets/zoo2.csv")
newdata

animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0       turtle     0         0     1     0         0        1         0   
1    chameleon     0         0     1     0         0        0         0   
2       iguana     0         0     1     0         0        0         1   
3       lizard     0         0     1     0         0        0         1   
4        gecko     0         0     1     0         0        0         0   
5       python     0         0     1     0         0        0         1   
6          boa     0         0     1     0         0        0         1   
7        adder     0         0     1     0         0        0         1   
8    crocodile     0         0     1     0         0        1         1   
9    alligator     0         0     1     0         0        1         1   
10     gharial     0         0     1     0         0        1         1   
11       skink     0         0     1     0         0        0         0   
12     anchovy     0         0     1     0         0        1         0   
13    flounder     0         0     1     0         0        1         0   
14     halibut     0         0     1     0         0        1         0   
15    mackerel     0         0     1     0         0        1         0   
16   barracuda     0         0     1     0         0        1         1   
17      marlin     0         0     1     0         0        1         0   
18       trout     0         0     1     0         0        1         0   
19  salamander     0         0     1     0         0        1         0   
20       siren     0         0     1     0         0        1         0   
21   tree frog     0         0     1     0         0        1         0   
22   dart frog     0         0     1     0         0        1         0   
23   firebelly     0         0     1     0         0        1         0   
24   wart toad     0         0     1     0         0        1         0   
25    mosquito     0         0     1     0         1        0         0   
26      hornet     0         0     1     0         1        0         1   
27     cricket     0         0     1     0         0        0         0   
28      beetle     0         0     1     0         0        0         0   
29   butterfly     1         0     1     0         1        0         0   
30    palmetto     0         0     1     0         1        0         1   
31   cockroach     0         0     1     0         0        0         0   
32      mantis     0         0     1     0         0        0         1   
33   dragonfly     0         0     1     0         1        0         0   
34       aphid     0         0     1     0         0        0         0   
35      cicada     0         0     1     0         1        0         0   
36     antlion     0         0     1     0         1        0         0   
37     scallop     0         0     1     0         0        1         0   
38      spider     0         0     1     0         0        0         1   
39       snail     0         0     1     0         0        0         0   
40    silkworm     0         0     1     0         0        0         0   
41   jellyfish     0         0     1     0         0        1         0   
42       squid     0         0     1     0         0        1         0   

    toothed  backbone  breathes  venomous  fins  legs  tail  domestic  \
0         0         1         1         0     0     4     1         1   
1         1         1         1         0     0     4     1         1   
2         1         1         1         0     0     4     1         1   
3         1         1         1         0     0     4     1         0   
4         1         1         1         0     0     4     1         1   
5         1         1         1         1     0     0     1         0   
6         1         1         1         0     0     0     1         0   
7         1         1         1         1     0     0     1         0   
8         1         1         1         0    

In [55]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(newdata.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['feathers', 'toothed', 'legs', 'airborne', 'backbone', 'tail', 'aquatic', 'hair']


In [56]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [57]:
sec_data

feathers  toothed  legs  airborne  backbone  tail  aquatic  hair  \
0          0        0     4         0         1     1        1     0   
1          0        1     4         0         1     1        0     0   
2          0        1     4         0         1     1        0     0   
3          0        1     4         0         1     1        0     0   
4          0        1     4         0         1     1        0     0   
5          0        1     0         0         1     1        0     0   
6          0        1     0         0         1     1        0     0   
7          0        1     0         0         1     1        0     0   
8          0        1     4         0         1     1        1     0   
9          0        1     4         0         1     1        1     0   
10         0        1     4         0         1     1        1     0   
11         0        1     4         0         1     1        0     0   
12         0        0     0         0         1     1        1     0   
13         0        0     0         0         1     1        1     0   
14         0        0     0         0         1     1        1     0   
15         0        0     0         0         1     1        1     0   
16         0        1     0         0         1     1        1     0   
17         0        0     0         0         1     1        1     0   
18         0        0     0         0         1     1        1     0   
19         0        1     4         0         1     1        1     0   
20         0        1     2         0         1     1        1     0   
21         0        1     4         0         1     0        1     0   
22         0        1     4         0         1     0        1     0   
23         0        1     4         0         1     0        1     0   
24         0        1     4         0         1     0        1     0   
25         0        0     6         1         0     0        0     0   
26         0        0     6         1         0     0        0     0   
27         0        0     6         0         0     0        0     0   
28         0        0     6         0         0     0        0     0   
29         0        0     6         1         0     0        0     1   
30         0        0     6         1         0     0        0     0   
31         0        0     6         0         0     0        0     0   
32         0        0     6         0         0     0        0     0   
33         0        0     6         1         0     0        0     0   
34         0        0     6         0         0     0        0     0   
35         0        0     6         1         0     0        0     0   
36         0        0     6         1         0     0        0     0   
37         0        0     0         0         0     0        1     0   
38         0        1     8         0         0     0        0     0   
39         0        0     0         0         0     0        0     0   
40         0        0     0         0         0     0        0     0   
41         0        0     0         0         0     0        1     0   
42         0        0     0         0         0     0        1     0   

    class_type  
0            3  
1            3  
2            3  
3            3  
4            3  
5            3  
6            3  
7            3  
8            3  
9            3  
10           3  
11           3  
12           4  
13           4  
14           4  
15           4  
16           4  
17           4  
18           4  
19           5  
20           5  
21           5  
22           5  
23           5  
24           5  
25           6  
26           6  
27           6  
28           6  
29           6  
30           6  
31           6  
32           6  
33           6  
34           6  
35           6  
36           6  
37           7  
38           7  
39           7  
40           7  
41           7  
42           7

In [61]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [62]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(train2), len(val2), len(test2))

NameError: name 'train2' is not defined

In [63]:
sec_train.head()

feathers  toothed  legs  airborne  backbone  tail  aquatic  hair  \
12         0        0     0         0         1     1        1     0   
27         0        0     6         0         0     0        0     0   
4          0        1     4         0         1     1        0     0   
11         0        1     4         0         1     1        0     0   
18         0        0     0         0         1     1        1     0   

    class_type  
12           4  
27           6  
4            3  
11           3  
18           4

In [64]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['feathers', 'toothed', 'legs', 'airborne',
                                  'backbone', 'tail', 'aquatic', 'hair'])])

In [65]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

feathers  toothed      legs  airborne  backbone  tail  aquatic  hair  \
0       0.0      0.0  0.000000       0.0       0.0   0.0      0.0   0.0   
1       0.0      1.0  1.333333       0.0       0.0   0.0      0.0   0.0   
2       0.0      0.0  0.000000       0.0       0.0   0.0      0.0   0.0   
3       0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0   
4       0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0   
5       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0   
6       0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0   
7       0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0   
8       0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0   

   class_type  
0         7.0  
1         7.0  
2         7.0  
3         6.0  
4         6.0  
5         6.0  
6         3.0  
7         3.0  
8         5.0

In [66]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models
#[MultinomialNB(), KNeighborsClassifier, DecisionTreeClassifier(), LogisticRegression, KNeighborsClassifier]

[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]

In [67]:
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

feathers  toothed      legs  airborne  backbone  tail  aquatic  hair
0        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
1        0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
2        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
3        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
4        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
5        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
6        0.0      1.0  0.000000       0.0       1.0   1.0      1.0   0.0
7        0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
8        0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
9        0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
10       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
11       0.0      1.0  0.000000       0.0       1.0   1.0      0.0   0.0
12       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   1.0
13       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
14       0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
15       0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0
16       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0
17       0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
18       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0
19       0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
20       0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
21       0.0      1.0  0.000000       0.0       1.0   1.0      0.0   0.0
22       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
23       0.0      0.0  0.666667       0.0       1.0   1.0      1.0   0.0
24       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0

In [68]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [69]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.5555555555555556
KNeighborsClassifier() 	 0.8857142857142857
SVC(kernel='sigmoid') 	 0.4
DecisionTreeClassifier() 	 0.8857142857142857
DecisionTreeClassifier(splitter='random') 	 0.8857142857142857


In [70]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

KNeighborsClassifier()

In [71]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.6944444444444444


In [72]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=40))

In [73]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
x_data2 = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x_data2

feathers  toothed      legs  airborne  backbone  tail  aquatic  hair
0        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
1        0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
2        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
3        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
4        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
5        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
6        0.0      1.0  0.000000       0.0       1.0   1.0      1.0   0.0
7        0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
8        0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
9        0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
10       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
11       0.0      1.0  0.000000       0.0       1.0   1.0      0.0   0.0
12       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   1.0
13       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
14       0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
15       0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0
16       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0
17       0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
18       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0
19       0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
20       0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
21       0.0      1.0  0.000000       0.0       1.0   1.0      0.0   0.0
22       0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
23       0.0      0.0  0.666667       0.0       1.0   1.0      1.0   0.0
24       0.0      0.0  0.000000       0.0       0.0   0.0      1.0   0.0
0        0.0      0.0  0.000000       0.0       0.0   0.0      0.0   0.0
1        0.0      1.0  1.333333       0.0       0.0   0.0      0.0   0.0
2        0.0      0.0  0.000000       0.0       0.0   0.0      0.0   0.0
3        0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
4        0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
5        0.0      0.0  1.000000       1.0       0.0   0.0      0.0   0.0
6        0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0
7        0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0
8        0.0      1.0  0.666667       0.0       1.0   0.0      1.0   0.0
0        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
1        0.0      0.0  1.000000       0.0       0.0   0.0      0.0   0.0
2        0.0      1.0  0.000000       0.0       1.0   1.0      0.0   0.0
3        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
4        0.0      1.0  0.333333       0.0       1.0   1.0      1.0   0.0
5        0.0      1.0  0.666667       0.0       1.0   1.0      1.0   0.0
6        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0
7        0.0      0.0  0.000000       0.0       1.0   1.0      1.0   0.0
8        0.0      1.0  0.666667       0.0       1.0   1.0      0.0   0.0

In [74]:
# Произведем кластеризацию
for model2 in k_models:
    model2.fit(x_data2)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (14) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (14) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x_data2)
    s = silhouette_score(x_data2, cluster_labels)
    sils.append(s)
    print(s)

0.5919274406206155
0.792835310084613
0.8837209302325582
0.7674418604651163


In [76]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=20)
0.8837209302325582
